In [ ]:
# Original example: https://github.com/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb

In [1]:
from autogen import config_list_from_json, UserProxyAgent, AssistantAgent
from datetime import datetime
import os, requests, json
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [2]:
# Pick up list of models to use
# Note; order of model matters
config_list = config_list_from_json(env_or_file = "../OAI_CONFIG_LIST")

In [3]:
BING_SUBSCRIPTION_KEY = os.environ['BING_SUBSCRIPTION_KEY']
HEADERS = {"Ocp-Apim-Subscription-Key": BING_SUBSCRIPTION_KEY}
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"

In [4]:
# Define functions
def bing_search(query: str) -> str | None:
    params = {"q": query, "count": 10}
    resp = requests.get(ENDPOINT, params=params, headers=HEADERS)
    if resp.status_code == 200:
        body = resp.json().get("webPages", {"value": []}).get("value")
        body = list(map(lambda item: {"name": item.get("name"), "url": item.get("url"), "snippet": item.get("snippet")}, body))
        formatting = []
        for item in body:
            formatting.append(
                "\n".join(["{key}: {value}\n".format(key=key, value=value) for key, value in item.items()])
            )
        return "\n\n".join(formatting)

In [6]:
print(bing_search(query="Sevilla")[0:1])

name: Seville - Wikipedia

url: https://en.wikipedia.org/wiki/Seville

snippet: Seville is the capital and largest city of Andalusia, the fourth-largest city in Spain and the 26th most populous municipality in the EU. It has a rich history as a Roman, Moorish, Christian and Spanish city, with a UNESCO World Heritage Site of three buildings: the Alcázar palace complex, the Cathedral and the General Archive of the Indies. It is also known as the \"Pearl of Andalusia\" for its cultural and economic achievements.


name: Sevilla FC - Wikipedia

url: https://en.wikipedia.org/wiki/Sevilla_FC

snippet: Sevilla FC is a Spanish professional football club based in Seville, the capital and largest city of the autonomous community of Andalusia. It plays in La Liga and has won the UEFA Cup/Europa League seven times, the most of any club. It is Spain's oldest sporting club devoted to football and the most successful in Andalusia.


name: THE 10 BEST Things to Do in Seville - Tripadvisor

url: https:

In [5]:
llm_config = {
    "functions": [
        {
            "name": "bing-search",
            "description": "Websearch using bing - will return title, snippet and url for the first 10 results",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "A search query",
                    }
                },
                "required": ["query"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

In [6]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10
    )

In [7]:
# register the functions
user_proxy.register_function(
    function_map={
        "bing-search": bing_search,
    }
)

In [8]:
chatbot = AssistantAgent(
    name="chatbot",
    system_message="For research tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

In [9]:
# start the conversation
user_proxy.initiate_chat(
    chatbot,
    message="What is the main focus of the meetup group 'IA Generativa Sevilla'?",
    clear_history=False
)

user_proxy (to chatbot):

What is the main focus of the group 'IA Generativa Sevilla'?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: bing-search *****
Arguments: 
{
  "query": "IA Generativa Sevilla main focus"
}
************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION bing-search...
user_proxy (to chatbot):

***** Response from calling function "bing-search" *****
name: Generative artificial intelligence - Wikipedia

url: https://en.wikipedia.org/wiki/Generative_artificial_intelligence

snippet: Artificial Intelligence is an idea that has been captivating society since the mid-20th century. It began with science fiction familiarizing the world with the concept but the idea wasn't fully seen in the scientific manner until Alan Tu